In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [3]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"Key exists and starts with {openai_api_key[:10]}")
else:
    print('There is an issue with the API Key or it might not exist')

MODEL = 'gpt-4.1-mini'
openai = OpenAI()

Key exists and starts with sk-proj-61


In [4]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [5]:
def chat(message, history):
    history = [{'role':h ['role'], 'content':h ['content']} for h in history]
    messages = [{'role': 'system', 'content': system_message}] + history + [{'role': 'user', 'content': message}]
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)
    result = ''
    for chunk in stream:
        result += chunk.choices[0].delta.content or ''
        yield result

In [6]:
gr.ChatInterface(fn=chat, type='messages').launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
